In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

resolution     = config.resolution
region         = config.region
data_path      = config.data_path
post_data_path = config.post_data_path
shp_path       = config.shp_path
fig_path       = config.fig_path

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

area_sum = 19.925158940748766

Entered /tera04/zhwei/xionghui/bedrock/run/0p1/


In [2]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),

    ('IGBP', 'LC', 0),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

print(df[df['IGBP']<10]['Area'].sum())

df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask1234'] == 1]


print(df['Area'].sum())

# df.drop(labels='Area',axis=1,inplace=True)
df.drop(labels='mask1234',axis=1,inplace=True)
df.drop(labels='Sbedrock',axis=1,inplace=True)
# with open('csv/Global_statistics.csv','w') as f:
#     df.to_csv(f)
df


64804308000000.0
9642207000000.0


,lat,lon,Area,IGBP
1261139,-54.95,-66.05,71010944.0,9
1394253,-51.25,-74.65,77395280.0,5
1448244,-49.75,-75.55,79893072.0,2
1495106,-48.45,-69.35,82013592.0,7
1509500,-48.05,-69.95,82657616.0,7
...,...,...,...,...
5881623,73.35,102.35,35431396.0,7
5885223,73.45,102.35,35224568.0,7
5906802,74.05,100.25,33981340.0,7
5921289,74.45,108.95,33150462.0,7


In [ ]:
df = pd.DataFrame()

with nc.Dataset('Sbedrock.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('Sbedrock', 'Sbedrock'),
    ('mask1234', 'Band1'),
    ('Area', 'area'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

shp2 = gpd.read_file(shp_path+'World_CN/ne_10m_admin_0_countries_chn.shp')
df = df.reset_index(drop=True)
gdf_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df['lon'], df['lat']), crs='EPSG:4326')
result2 = gpd.sjoin(gdf_points, shp2, how='left', predicate='within')

df['Sovereignt_short'] = result2['ISO_A3']
df = df[df['Sovereignt_short']=='Brazil']

print(df['Area'].sum()/(1e12))

df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask1234'] == 1]
df.drop(labels='mask1234',axis=1,inplace=True)

print(df['Area'].sum()/(1e12))

# with open('csv/Global.csv','w') as f:
#     df.to_csv(f)

# df1 = df[df['Sovereignt_short'] == 'USA']
# # print(df1)

# with open('csv/US.csv','w') as f:
#     df1.to_csv(f)

# df1
df